In [4]:
"""
convert currencies using foreign exchange rates
"""
import pandas as pd
import wrds
from datetime import datetime

%matplotlib inline
import matplotlib.pyplot as plt

from dotenv import load_dotenv
load_dotenv("./../.env")

###################
# Connect to WRDS #
###################
conn=wrds.Connection()


WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [9]:
import os
DROPBOX_PATH = os.getenv("DATA_FOLDER")
print(DROPBOX_PATH)


# load unique curcd list
curcds = pd.read_csv(f"{DROPBOX_PATH}/raw_data/compustat/unique_curcd.csv")['curcd']

/Users/boyie/Library/CloudStorage/Dropbox/Fortune500_SDG_Analysis/data


In [10]:
print(curcds)

0     AUD
1     BRL
2     CHF
3     CNY
4     EUR
5     GBP
6     HKD
7     INR
8     JPY
9     KRW
10    MXN
11    MYR
12    NOK
13    RUB
14    SAR
15    SEK
16    SGD
17    THB
18    TRY
19    TWD
20    UGX
21    USD
Name: curcd, dtype: object


In [23]:
def fetch_exchange_rate(conn, _from = 'CNY' , _to = 'USD'):

    # get _from -- GBP rate
    _from_gbp = conn.raw_sql(f"""
                             SELECT tocurm, fromcurm, datadate, exratm 
                             FROM comp.g_exrt_mth 
                             WHERE tocurm='{_from}'
                             """, 
                        date_cols=['datadate'])

    # get _to -- GBP rate
    _to_gbp = conn.raw_sql(f"""
                             SELECT tocurm, fromcurm, datadate, exratm 
                             FROM comp.g_exrt_mth 
                             WHERE tocurm='{_to}'
                             """, 
                        date_cols=['datadate'])

    # rename columns for later merging
    _from_gbp.rename(columns={'exratm':f'{_from}_gbp'}, inplace=True)
    _to_gbp.rename(columns={'exratm':f'{_to}_gbp'}, inplace=True)

    # merge to create exchange rate between CNY and USD
    _from_to = pd.merge(_from_gbp[['datadate',f'{_from}_gbp']], 
                        _to_gbp[['datadate',f'{_to}_gbp']], 
                        how='inner', 
                        on = 'datadate')

    # create USD CNY rate
    _from_to[f'{_from}_{_to}'] = _from_to[f'{_from}_gbp'] / _from_to[f'{_to}_gbp']

    return _from_to[['datadate', f'{_from}_{_to}']]





In [24]:
# Initialize an empty dataframe to store all exchange rates
exchange_rates_df = pd.DataFrame()

# Iterate over each currency code and fetch the exchange rate to USD
for curcd in curcds:
    if curcd != 'USD':  # Skip USD as it doesn't need conversion
        exchange_rate = fetch_exchange_rate(conn, _from=curcd, _to='USD')
        if exchange_rates_df.empty:
            exchange_rates_df = exchange_rate
        else:
            exchange_rates_df = pd.merge(exchange_rates_df, exchange_rate, on='datadate', how='outer')

# Save the combined exchange rate table to a CSV file
exchange_rates_df.to_csv(f"{DROPBOX_PATH}/cleaned_data/exchange_rates_to_usd.csv", index=False)

print("Exchange rate table has been successfully created and saved.")

Exchange rate table has been successfully created and saved.


In [25]:
exchange_rates_df

,datadate,AUD_USD,BRL_USD,CHF_USD,CNY_USD,EUR_USD,GBP_USD,HKD_USD,INR_USD,JPY_USD,...,MYR_USD,NOK_USD,RUB_USD,SAR_USD,SEK_USD,SGD_USD,THB_USD,TRY_USD,TWD_USD,UGX_USD
0,1982-01-31,0.898538,NaN,1.846173,NaN,NaN,0.530790,5.802102,10.446165,264.862626,...,2.259915,5.868572,NaN,3.924094,5.626614,2.062905,23.074666,NaN,NaN,NaN
1,1982-02-28,0.931532,NaN,1.899000,NaN,NaN,0.549451,5.895000,9.310001,237.250389,...,2.312000,6.023500,NaN,4.062050,5.797500,2.114000,23.050004,NaN,NaN,NaN
2,1982-03-31,0.952925,NaN,1.938500,NaN,NaN,0.561482,5.843000,9.350001,248.250173,...,2.339000,6.106000,NaN,3.853064,5.937000,2.130500,23.049999,NaN,NaN,NaN
3,1982-04-30,0.942152,NaN,1.957000,NaN,NaN,0.557258,5.817000,9.350000,235.849570,...,2.302000,5.972500,NaN,3.820588,5.795000,2.102000,23.000001,NaN,NaN,NaN
4,1982-05-31,0.952109,NaN,1.997500,NaN,NaN,0.558347,5.747500,9.389999,243.300240,...,2.290000,6.023500,NaN,3.891488,5.850000,2.090000,22.999999,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,2024-04-30,1.539880,5.174611,0.916647,7.238084,0.935170,0.798403,7.821158,83.466173,157.446782,...,4.767824,11.071058,93.545918,3.749860,10.983313,1.363433,37.152727,32.395851,32.610939,3814.635391
508,2024-05-31,1.506052,5.241434,0.903489,7.241984,0.922116,0.785916,7.820025,83.455774,157.214090,...,4.706932,10.513360,90.373443,3.750629,10.534030,1.351855,36.824353,32.214005,32.522396,3811.057989
509,2024-06-30,1.498813,5.551590,0.898560,7.266814,0.933296,0.791264,7.808197,83.359261,160.865014,...,4.717360,10.656354,86.247466,3.751701,10.598828,1.355594,36.735711,32.769662,32.485761,3706.851122
510,2024-07-31,1.532046,5.648470,0.879994,7.219999,0.923993,0.778756,7.811541,83.663388,150.470207,...,4.594424,10.925239,85.950710,3.751966,10.719960,1.337513,35.563349,33.108553,32.710376,3721.117874
